# Single Responsibility Principle

"A class should have one, and only one, responsibility and reason to change"

This principle states that a class should have one and only one responsibility and reason to change.

* Database code in Video class (Violates Single Responsibility Principle)

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "mysql+mysqlconnector://root:@localhost/youtube"

engine = create_engine(DATABASE_URL)
Base = declarative_base()

class Video(Base):
    __tablename__ = "Video"

    id = Column(Integer, primary_key=True, autoincrement=True)
    title = Column(String(255))
    time = Column(Integer)
    likes = Column(Integer)
    views = Column(Integer)

    def __init__(self, title, time, likes, views):
        self.title = title
        self.time = time
        self.likes = likes
        self.views = views
        self.engine = create_engine(DATABASE_URL)
        Base.metadata.create_all(self.engine)
        Session = sessionmaker(bind=self.engine)
        self.session = Session()

    def get_number_of_hours_played(self):
        return (self.time / 3600.0) * self.views

    def persist(self):
        try:
            self.session.add(self)
            self.session.commit()
            print("Video persisted successfully.")
        except Exception as e:
            self.session.rollback()
            print(f"Error persisting video: {e}")
        finally:
            self.session.close()

* Applying Single Responsibility Principle

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "mysql+mysqlconnector://root:@localhost/youtube"

engine = create_engine(DATABASE_URL)
Base = declarative_base()

class Video(Base):
    __tablename__ = "Video"

    id = Column(Integer, primary_key=True, autoincrement=True)
    title = Column(String(255))
    time = Column(Integer)
    likes = Column(Integer)
    views = Column(Integer)

    def __init__(self, title, time, likes, views):
        self.title = title
        self.time = time
        self.likes = likes
        self.views = views

    def get_number_of_hours_played(self):
        return (self.time / 3600.0) * self.views

class DatabaseConnection:
    def __init__(self, db_url):
        self.engine = create_engine(db_url)
        Base.metadata.create_all(self.engine)
        Session = sessionmaker(bind=self.engine)
        self.session = Session()

    def get_session(self):
        return self.session

    def close_session(self):
        self.session.close()


class VideoDAO:
    def __init__(self, db_connection):
        self.db_connection = db_connection
        self.session = db_connection.get_session()

    def persist(self, video):
        try:
            self.session.add(video)
            self.session.commit()
            print("Video persisted successfully.")
        except Exception as e:
            self.session.rollback()
            print(f"Error persisting video: {e}")
        finally:
            self.db_connection.close_session()

### Single Responsibility Principle (SRP) Explanation

The Single Responsibility Principle (SRP) states that a class should have only one reason to change, meaning it should have only one job or responsibility.

**First Code (Violation of SRP):**

The initial `Video` class had two primary responsibilities:

1.  **Data Representation:** Holding video attributes like `title`, `time`, `likes`, and `views`.
2.  **Database Interaction:** Managing database connection, creating tables, and persisting video data.

**Problem:** The class also handles database creation, which should be done by separate class or class only dealing with database stuff.

**Why is this bad?**

*   If the database connection details (like the URL) need to change, or if the persistence logic needs to be updated (e.g., switching to a different ORM), you'd have to modify the `Video` class. This change might unintentionally affect the video data representation logic.
*   Testing becomes more complex because you can't easily test the video data representation logic in isolation from the database interaction logic.

**Second Code (Adherence to SRP):**

The second code demonstrates better adherence to the SRP by separating responsibilities into distinct classes:

*   **`Video` Class:** *Solely* responsible for representing video data and calculating `get_number_of_hours_played`. It doesn't deal with database interactions.
*   **`DatabaseConnection` Class:** Manages the database connection, engine creation, and session management.
*   **`VideoDAO` Class:** Handles the data access operations (persistence) for the `Video` class.

**Solution:**
* Separate class for each responsibility.

**Why is this better?**

*   Each class has a single, well-defined purpose.
*   Changes to the database connection or persistence logic only affect the `DatabaseConnection` or `VideoDAO` classes, without impacting the `Video` class.
*   Testing becomes easier because each class can be tested independently.
*   Makes code more readable, easier to maintain and modify.

## Open/Closed Principle (OCP)

**Definition:** A class should be open for extension, but closed for modification.

In essence, you should be able to add new functionality to a class *without* changing the class itself.

**SOLID**

`"A class should be open for extension, but closed for modification"`

In [ ]:
class Video:
    def __init__(self, title, time, likes, views, category):
        self.title = title
        self.time = time
        self.likes = likes
        self.views = views
        self.category = category

    def get_number_of_hours_played(self):
        return (self.time / 3600.0) * self.views

    def get_category(self):
        return self.category

In [ ]:
class EarningsCalculator:
    def calculate_earnings(self, video):
        if video.get_category() == "EDUCATIONAL":
            return self.calculate_educational_earnings(video)
        elif video.get_category() == "GAMING":
            return self.calculate_gaming_earnings(video)
        elif video.get_category() == "ENTERTAINMENT":
            return self.calculate_entertainment_earnings(video)
        else:
            return 0

    def calculate_educational_earnings(self, video):
        return video.likes * 0.013 + video.views * 0.0013

    def calculate_gaming_earnings(self, video):
        return video.likes * 0.012 + video.views * 0.0012

    def calculate_entertainment_earnings(self, video):
        return video.likes * 0.011 + video.views * 0.0011

## OCP Violation & Solution

**Problem:**

The initial `EarningsCalculator` directly checks the video's category using `if/elif/else` statements.

This violates the Open/Closed Principle because:

*   **Closed for Modification Violation:** If you want to add a new video category (e.g., "SPORTS"), you *must* modify the `EarningsCalculator` class by adding another `elif` condition.
*   **Tight Coupling:** The `EarningsCalculator` is tightly coupled to the specific video categories.

**Solution:**

Use inheritance and interfaces/abstract base classes to create a more flexible design.

In [ ]:
from abc import ABC, abstractmethod

class IEarningsCalculator(ABC):
    @abstractmethod
    def calculate_earnings(self, video):
        pass


class EducationalEarningsCalculator(IEarningsCalculator):
    def calculate_earnings(self, video):
        return video.likes * 0.013 + video.views * 0.0013


class GamingEarningsCalculator(IEarningsCalculator):
    def calculate_earnings(self, video):
        return video.likes * 0.012 + video.views * 0.0012


class EntertainmentEarningsCalculator(IEarningsCalculator):
    def calculate_earnings(self, video):
        return video.likes * 0.011 + video.views * 0.0011


class EarningsCalculator:
    def __init__(self, category_calculators):
        self.category_calculators = category_calculators

    def calculate_earnings(self, video):
        calculator = self.category_calculators.get(video.get_category())
        if calculator:
            return calculator.calculate_earnings(video)
        else:
            return 0

## Liskov Substitution Principle (LSP)

**Definition:** Subtypes should be replaceable by their base types.

In essence, if you have a base class and a derived class, you should be able to use the derived class anywhere you're using the base class without unexpected behavior or errors.

**SOLID**

`"Subtypes should be replaceable by their base types"`

In [ ]:
class Video:
    def __init__(self, title, time, likes, views):
        self.title = title
        self.time = time
        self.likes = likes
        self.views = views

    def get_number_of_hours_played(self):
        return (self.time / 3600.0) * self.views

    def play_random_ad(self):
        print("Playing a random ad.")


class PremiumVideo(Video):
    def __init__(self, title, time, likes, views, premium_id):
        super().__init__(title, time, likes, views)
        self.premium_id = premium_id

    def play_random_ad(self):
        raise Exception("No ads play during premium videos")

### LSP Violation & Solution

**Problem (LSP Violation):**

The `PremiumVideo` class inherits from `Video` but overrides the `play_random_ad` method to throw an exception.

This violates the Liskov Substitution Principle because:

*   If you have a function that expects a `Video` object and calls `play_random_ad()`, it will work fine for regular `Video` objects.
*   However, if you pass a `PremiumVideo` object to the same function, it will throw an exception, leading to unexpected behavior and potentially crashing the application.
*   `PremiumVideo` is **not** a fully substitutable replacement for `Video`.

**Why is this bad?**

It breaks the contract defined by the base class, and any code relying on that contract may fail when dealing with the derived class. It makes code harder to reason about and more prone to errors.

**Solution:**

In [ ]:
class VideoManager:
    def __init__(self, title, time, likes, views):
        self.title = title
        self.time = time
        self.likes = likes
        self.views = views

    def get_number_of_hours(self):
        return (self.time / 3600.0) * self.views

    def play_random_ad(self):
        print("Playing a random ad.")


class Video:
    def __init__(self, manager):
        self.manager = manager

    def get_number_of_hours_played(self):
        return self.manager.get_number_of_hours()

    def play_random_ad(self):
        self.manager.play_random_ad()


class PremiumVideo:
    def __init__(self, manager, premium_id):
        self.manager = manager
        self.premium_id = premium_id

    def get_number_of_hours_played(self):
        return self.manager.get_number_of_hours()

## Interface Segregation Principle (ISP)

**Definition:** Many specific interfaces are better than a general interface.

In essence, clients should not be forced to depend upon interfaces that they do not use. Break large interfaces into smaller, more specific ones so that clients only need to know about the methods that are of interest to them.

**SOLID**

`"Many specific interfaces are better than a general interface"`

In [ ]:
from abc import ABC, abstractmethod

class IVideoActions(ABC):
    @abstractmethod
    def get_number_of_hours_played(self):
        pass

    @abstractmethod
    def play_random_ad(self):
        pass

class Video(IVideoActions):
    def __init__(self, title, time, likes, views):
        self.title = title
        self.time = time
        self.likes = likes
        self.views = views

    def get_number_of_hours_played(self):
        return (self.time / 3600.0) * self.views

    def play_random_ad(self):
        print("Playing a random ad.")


class PremiumVideo(IVideoActions):
    def __init__(self, title, time, likes, views, premium_id):
        self.title = title
        self.time = time
        self.likes = likes
        self.views = views
        self.premium_id = premium_id

    def get_number_of_hours_played(self):
        return (self.time / 3600.0) * self.views

    def play_random_ad(self):
        print("No ads play during premium videos.")

### ISP Violation & Solution

**Problem:**

The `IVideoActions` interface has two methods: `get_number_of_hours_played()` and `play_random_ad()`. Both `Video` and `PremiumVideo` implement this interface.

However, `PremiumVideo` doesn't actually play ads. It has to provide an empty implementation or a placeholder implementation, forcing the class to implement method not using them which violates ISP.

**Why is this bad?**

*   Clients that use `PremiumVideo` are forced to depend on the `play_random_ad()` method, even though they don't need it.
*   It bloats the interface and makes it less cohesive.
*   It may introduce confusion or unexpected behavior if `play_random_ad()` is called on a `PremiumVideo` object.

**Solution:**

Segregate the interface into smaller, more specific interfaces.

In [ ]:
from abc import ABC, abstractmethod

class IVideoActions(ABC):
    @abstractmethod
    def get_number_of_hours_played(self):
        pass

class IAdsActions(ABC):
    @abstractmethod
    def play_random_ad(self):
        pass

class Video(IVideoActions,IAdsActions):
    def __init__(self, title, time, likes, views):
        self.title = title
        self.time = time
        self.likes = likes
        self.views = views

    def get_number_of_hours_played(self):
        return (self.time / 3600.0) * self.views

    def play_random_ad(self):
        print("Playing a random ad.")

class PremiumVideo(IVideoActions):
    def __init__(self, title, time, likes, views, premium_id):
        self.title = title
        self.time = time
        self.likes = likes
        self.views = views
        self.premium_id = premium_id

    def get_number_of_hours_played(self):
        return (self.time / 3600.0) * self.views

## Dependency Inversion Principle (DIP)

**Definition:**
1. High-level modules should not depend on low-level modules. Both should depend on abstractions (interfaces or abstract classes).
2. Abstractions should not depend on details. Details (concrete implementations) should depend on abstractions.

In essence, depend upon abstractions, not concretions.

**SOLID**

`"We must depend on abstractions and not concrete classes"`

In [ ]:
from abc import ABC, abstractmethod

class IEarningsCalculator(ABC):
    @abstractmethod
    def calculate_earnings(self, video):
        pass

class EducationalEarningsCalculator(IEarningsCalculator):
    def calculate_earnings(self, video):
        return video.likes * 0.013 + video.views * 0.0013

class GamingEarningsCalculator(IEarningsCalculator):
    def calculate_earnings(self, video):
        return video.likes * 0.012 + video.views * 0.0012

class EntertainmentEarningsCalculator(IEarningsCalculator):
    def calculate_earnings(self, video):
        return video.likes * 0.011 + video.views * 0.0011

class Service:
    def __init__(self, calculator: IEarningsCalculator):
        self.calculator = calculator

    def calculate_earnings(self, video):
        return self.calculator.calculate_earnings(video)

### DIP Explanation

In the given code, the **Dependency Inversion Principle (DIP)** is well-applied. The principle has two parts:

1.  **High-level modules should not depend on low-level modules. Both should depend on abstractions.**
2.  **Abstractions should not depend on details. Details should depend on abstractions.**

**How DIP is implemented in this example:**

*   **High-Level Module:** `Service` class is a high-level module. It defines the business logic of calculating earnings. It orchestrates the calculation but doesn't know the specific implementation details.
*   **Low-Level Modules:** `EducationalEarningsCalculator`, `GamingEarningsCalculator`, and `EntertainmentEarningsCalculator` are low-level modules. They provide concrete implementations of earnings calculation for different video categories.
*   **Abstraction:** `IEarningsCalculator` is the abstraction. It's an interface that defines the `calculate_earnings()` method. Both high-level (`Service`) and low-level (`*EarningsCalculator`) modules depend on this abstraction.
*   **Dependency Inversion:**  Instead of `Service` directly depending on concrete `*EarningsCalculator` classes, it depends on the `IEarningsCalculator` interface. The concrete implementations (e.g., `EducationalEarningsCalculator`) also depend on the `IEarningsCalculator` interface. This "inverts" the traditional dependency structure, where high-level modules would typically depend on low-level modules.

**Benefits of DIP (and how it's demonstrated in this code):**

1.  **Decoupling:** The `Service` class is decoupled from the specific earnings calculation implementations. You can change the earnings calculation logic for one video category without affecting the `Service` class.
2.  **Flexibility:** You can easily add new video categories and corresponding earnings calculators without modifying the `Service` class. Just create a new class that implements `IEarningsCalculator` and inject it into the `Service`.
3.  **Testability:** You can easily test the `Service` class by injecting mock implementations of `IEarningsCalculator`. This allows you to isolate the `Service`'s logic from the complexities of the concrete earnings calculation implementations.
4.  **Maintainability:** The code is more maintainable because it's easier to understand and modify the different parts independently.